In [75]:
import rdflib

Load ontology and example

In [76]:
g = rdflib.Graph()
g.parse("../ontology/server.ttl", format='turtle')
g.parse("../ontology/library.ttl", format='turtle')
g.parse("../ontology/platform.ttl", format='turtle')

g.parse("../examples/server.ttl", format='turtle')
g.parse("../examples/library.ttl", format='turtle')
g.parse("../examples/platform.ttl", format='turtle')
g.parse("../examples/netconf-monit.ttl", format='turtle')
g.parse("../examples/catalog-ietf.ttl", format='turtle')
g.parse("../examples/catalog-oc.ttl", format='turtle')
print(len(g))

96438


Load prefixes

In [77]:
prefixes = """
PREFIX ndc-lb: <https://w3id.org/network-data-catalog/library#>
PREFIX ndc-pm: <https://w3id.org/network-data-catalog/platform#>
PREFIX ndc-sr: <https://w3id.org/network-data-catalog/server#>
PREFIX mod: <https://w3id.org/mod#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX xml: <http://www.w3.org/XML/1998/namespace>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX core: <https://ontology.unifiedcyberontology.org/uco/core/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX vann: <http://purl.org/vocab/vann/>
PREFIX schema: <http://schema.org/>
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX observable: <https://ontology.unifiedcyberontology.org/uco/observable/>
BASE <http://example.org/>
"""

## Step 1: Identifying available datastores

In [78]:
q_res = g.query(prefixes + """
    SELECT DISTINCT ?datastore ?type
    WHERE {
        ?device a ndc-sr:NetworkDevice ;
            ndc-sr:deviceId "srl" ;
            ndc-sr:implementsYangDatastore ?datastore .
        ?datastore rdf:type ?type .
    }
""")

for row in q_res:
    print(f"Datastore: {row.datastore} Type: {row.type}")

Datastore: http://example.org/NetworkDevice/srl/Datastore/intended Type: https://w3id.org/network-data-catalog/server#IntendedDatastore
Datastore: http://example.org/NetworkDevice/srl/Datastore/candidate Type: https://w3id.org/network-data-catalog/server#CandidateDatastore
Datastore: http://example.org/NetworkDevice/srl/Datastore/startup Type: https://w3id.org/network-data-catalog/server#StartupDatastore
Datastore: http://example.org/NetworkDevice/srl/Datastore/running Type: https://w3id.org/network-data-catalog/server#RunningDatastore
Datastore: http://example.org/NetworkDevice/srl/Datastore/operational Type: https://w3id.org/network-data-catalog/server#OperationalDatastore


## Step 2: Identifying implemented YANG modules for interface management

In [79]:
q_res = g.query(prefixes + """
    SELECT  ?moduleName ?revisionDate ?moduleDescription
    WHERE {
        <http://example.org/NetworkDevice/srl/Datastore/operational> ndc-lb:supportsDatastoreSchema ?schema .
        ?schema ndc-lb:usesModuleSet ?moduleSet .
        ?moduleSet ndc-lb:moduleImplementation ?moduleImplementation .
        ?moduleImplementation ndc-lb:implementedModule ?implementedModule .
        ?implementedModule ndc-lb:moduleName ?moduleName ;
                           ndc-lb:revisionDate ?revisionDate ;
                           ndc-lb:moduleDescription ?moduleDescription .
        FILTER(CONTAINS(STR(?moduleDescription), "interface"))
    }
""")

for row in q_res:
    print(f"Module: {row.moduleName}:{row.revisionDate} Description: {row.moduleDescription}")

Module: openconfig-interfaces:2023-11-06 Description: Model for managing network interfaces and subinterfaces.  This
module also defines convenience types / groupings for other
models to create references to interfaces:

 base-interface-ref (type) -  reference to a base interface
 interface-ref (grouping) -  container for reference to a
   interface + subinterface
 interface-ref-state (grouping) - container for read-only
   (opstate) reference to interface + subinterface

This model reuses data items defined in the IETF YANG model for
interfaces described by RFC 7223 with an alternate structure
(particularly for operational state data) and with
additional configuration items.

Portions of this code were derived from IETF RFC 7223.
Please reproduce this note if possible.

IETF code is subject to the following copyright and license:
Copyright (c) IETF Trust and the persons identified as authors of
the code.
All rights reserved.

Redistribution and use in source and binary forms, with or 

## Step 3 : Identifying dependencies for target YANG module

In [ ]:
q_res = g.query(prefixes + """
    SELECT ?moduleName ?revisionDate
    WHERE {
        <http://example.org/Module/openconfig-interfaces/2023-11-06>
            ?dependencyType+ ?dependency .
        ?dependency ndc-lb:moduleName ?moduleName ;
                    ndc-lb:revisionDate ?revisionDate .
    }
""")

for row in q_res:
    print(f"Module: {row.moduleName}:{row.revisionDate} ")

ParseException: Expected SelectQuery, found '<'  (at char 838), (line:20, col:9)

## Step 4: Collecting MDT protocol information.

In [ ]:
q_res = g.query(prefixes + """
    SELECT ?type ?addressValue ?username ?password
    WHERE {
        ?server rdf:type/rdfs:subClassOf* ndc-sr:YangServer ;
            rdf:type ?type ;
            ndc-sr:socketAddress ?socketAddress ;
            ndc-sr:serverAccount ?serverAccount .
        ?socketAddress observable:addressValue ?addressValue .
        ?serverAccount ndc-sr:username ?username ;
            core:hasFacet ?accountAuthenticationFacet .
        ?accountAuthenticationFacet observable:password ?password .
    }
""")

for row in q_res:
    print(f"Protocol: {row.type} Address: {row.addressValue} Username: {row.username} Password: {row.password}")

Protocol: https://w3id.org/network-data-catalog/server#GnmiServer Address: 172.100.100.11:57400 Username: admin Password: NokiaSrl1!
Protocol: https://w3id.org/network-data-catalog/server#NetconfServer Address: 172.100.100.11:830 Username: admin Password: NokiaSrl1!


## Step 5: Collecting NETCONF capabilities

In [83]:
q_res = g.query(prefixes + """
    SELECT ?netconfCapability
    WHERE {
        <http://example.org/NetworkDevice/srl/NetconfServer> rdf:type ndc-sr:NetconfServer ;
            ndc-sr:netconfCapability ?netconfCapability .
    }
""")

for row in q_res:
    print(f"Capability: {row.netconfCapability}")

Capability: https://w3id.org/network-data-catalog/server#ConfirmedCommitCapability
Capability: https://w3id.org/network-data-catalog/server#RollbackOnErrorCapability
Capability: https://w3id.org/network-data-catalog/server#CandidateCapability
Capability: https://w3id.org/network-data-catalog/server#StartupCapability
Capability: https://w3id.org/network-data-catalog/server#YangLibrary11Capability
Capability: https://w3id.org/network-data-catalog/server#UrlCapability
Capability: https://w3id.org/network-data-catalog/server#ValidateCapability
